# TP MLP PyTorch 
Dans ce TP, vous allez implémenter un réseau de neurones de type *Perceptron Multicouche* en utilisant la bibliothèque PyTorch.

## Configuration
### Si vous utilisez un ordinateur de l'Enseirb:
#### 1) Lancer une session linux (et non pas windows)
#### 2) Aller dans "Applications", puis "Autre", puis "conda_pytorch" (un terminal devrait s'ouvrir)
#### 3) Dans ce terminal, taper la commande suivante pour lancer Spyder :  
`spyder &`  
### Si vous utilisez votre ordinateur personnel, il faudra installer Spyder.  

---
---
## Dans tous les cas, ne pas oublier de configurer Spyder en suivant ces [instructions](https://gbourmaud.github.io/files/configuration_spyder_annotated.pdf).
---
---

# I) Introduction à PyTorch
La documentation de la bibliothèque PyTorch est [ici](https://pytorch.org/docs/1.12/ ). Il est également possible d'accéder à la documentation d'une fonction en tapant `help(torch.nom_de_la_fonction)` dans le terminal python de Spyder (exemple : `help(torch.matmul)` après avoir importé la biblitohèque PyTorch `import torch`).

### Créer un nouveau script python et copier/coller le code suivant :

In [1]:
import torch 

### Fonctionnalité "autograd" de PyTorch

Un `torch.tensor` est l'équivalent en PyTorch d'un `numpy.array` en Numpy : il s'agit d'un tableau multidimensionnel. 

In [ ]:
#Exemple de création d'un tableau bidimensionnel
x = torch.tensor([[1, 2, 3],[4, 5 ,6], [7, 8 ,9], [10, 11, 12]])
print(x.shape)

La principale différence entre un `numpy.array` et un `torch.tensor` est le fait que le `torch.tensor` permet l'utilisation de la fonctionnalité *autograd* (option `requires_grad=True`). 

In [ ]:
#Exemple de création d'un tableau bidimensionnel en activant l'autograd
x = torch.tensor([[1., 2., 3.],[4., 5. ,6.], [7., 8. ,9.], [10., 11., 12.]],requires_grad=True)
print(x)

Lorsque cette fonctionnalité est activée pour un `torch.tensor` $X$ , un graphe de calcul se crée et chaque opération faisant intervenir (directement ou indirectement) $X$ est ajoutée à ce graphe de calcul. Tout ce processus est transparent pour l'utilisateur. Ce processus de création d'un graphe de calcul correspond à l'étape de **propagation avant** vue en cours.  Lorsque la méthode `.backward()` d'une variable `torch.tensor` $y$ **scalaire** du graphe de calcul, est exécutée, l'étape de **rétropropagation** s'effectue et calcule la dérivée $\frac{\partial y}{\partial X}$. Le résultat est stockée dans le champs `x.grad`.

In [ ]:
#Exemple d'utilisation de l'autograd avec des scalaires

# Création des tenseurs scalaires
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2.)
b = torch.tensor(3.)

print(x.grad) # None

# Construction du graphe de calcul (propagation avant)
z = w*x
y = z+b # y = 2*x + 3

# Calcul du gradient (rétropropagation)
y.backward(torch.tensor(1.))

# Affichage du gradient
print(x.grad)    # x.grad = 2 

**Dessiner (sur papier) le graphe de calcul de l'exemple précédent.** Dans ce graphe, vous remarquerez qu'il y a trois *feuilles* : le tenseur $x$, le tenseur $w$ et le tenseur $b$. Pour obtenir  $\frac{\partial y}{\partial w}$ et $\frac{\partial y}{\partial b}$ en plus de $\frac{\partial y}{\partial x}$, il suffit d'utiliser l'option `requires_grad=True` sur $w$ et $b$.

In [ ]:
#Exemple d'utilisation de l'autograd avec des scalaires

# Création des tenseurs scalaires
x = torch.tensor(4., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

print(x.grad) # None
print(w.grad) # None
print(b.grad) # None

# Construction du graphe de calcul (propagation avant)
z = w*x
y = z+b

# Calcul des gradients (rétropropagation)
y.backward(torch.tensor(1.))

# Affichage des gradients
print(x.grad)    # x.grad = 2.
print(w.grad)    # w.grad = 4. 
print(b.grad)    # b.grad = 1. 

Ce mécanisme d'autograd fonctionne de la même manière lorsque les feuilles sont des tableaux multimensionnels plutôt que des scalaires.

In [ ]:
#Exemple d'utilisation de l'autograd avec des tenseurs 2D

# Création des tenseurs
X = torch.tensor([[1., 2., 3.],[4., 5. ,6.], [7., 8. ,9.], [10., 11., 12.]],requires_grad=True) #tableau 4x3 
W = torch.tensor([[1., 2.],[4., 5.], [7., 8.]],requires_grad=True) #tableau 3x2 
b = torch.tensor([4., 5.], requires_grad=True) #vecteur de taille 2


# Construction du graphe de calcul (propagation avant)
z1 = X.matmul(W)
z2 = z1 + b
y = z2.sum()

# Calcul des gradients (rétropropagation)
y.backward(torch.tensor(1.))

# Affichage des gradients
print(X.grad)
print(W.grad)
print(b.grad)


**Dessiner (sur papier) le graphe de calcul de l'exemple précédent.** Quelle devrait-être les tailles des variables `X.grad`, `W.grad` et `b.grad` ? Vérifier leurs tailles dans le code (attribut `.shape` d'un tenseur).

Comme nous venons de la voir, la fonctionnalité *autograd* est une implémentation du théorème de dérivation d'une fonction composée. Pour s'en convaincre, prenons le cas de la composition de deux fonctions $y=g(Z)$ et $Z=f(X)$. Nous allons comparer deux utilisations de l'autograd :

Cas 1) Calculer directement $\frac{\partial y}{\partial X}$ avec l'autograd

Cas 2) Calculer manuellement $\frac{\partial y}{\partial Z}$ (variable `dy_dZ`) et fournir ce gradient à l'autograd (`Z.backward(dy_dZ)`) pour qu'il termine le calcule de $\frac{\partial y}{\partial X}$.

Les gradients obtenus avec le cas 1 et le cas 2 doivent être parfaitement identiques.

In [ ]:
# CAS 1

# Création des tenseurs
X = torch.tensor([[1., 2., 3.],[4., 5. ,6.], [7., 8. ,9.], [10., 11., 12.]],requires_grad=True) #tableau 4x3 
W = torch.tensor([[1., 2.],[4., 5.], [7., 8.]],requires_grad=True) #tableau 3x2 

# Construction du graphe de calcul (propagation avant)
Z = X.matmul(W)
y = Z.sum()

y.backward(torch.tensor(1.))

print(X.grad)
# CAS 2

# Création des tenseurs
X = torch.tensor([[1., 2., 3.],[4., 5. ,6.], [7., 8. ,9.], [10., 11., 12.]],requires_grad=True) #tableau 4x3 
W = torch.tensor([[1., 2.],[4., 5.], [7., 8.]],requires_grad=True) #tableau 3x2 

# Construction du graphe de calcul (propagation avant)
Z = X.matmul(W)
y = Z.sum()

dy_dZ = torch.ones(Z.shape) #dérivée de la fonction sum
Z.backward(dy_dZ)

print(X.grad)

# II) Du MLP en Numpy au MLP en Pytorch sans *autograd*

Télécharger l'implémentation du MLP **en PyTorch** [main_MLP_two_layers_pytorch_without_autograd.py](https://gbourmaud.github.io/files/intro_deep_learning/TP/TP_MLP/misc/main_MLP_two_layers_pytorch_without_autograd.py) ainsi que le fichier [utils.py](https://gbourmaud.github.io/files/intro_deep_learning/TP/TP_MLP/misc/utils.py). Vérifier que le code fonctionne (le résultat devrait être quasi-identique à celui du TP précédent).  
  
Vous remarquerez que la bibliothèqe Numpy n'est plus utilisée, et à été entièrement remplacée par PyTorch.

**Il n'y a pas de travail à effectuer dans cette partie.**

# III) Utilisation de la fonctionnalité *autograd*

L'objectif de cette partie consiste à remplacer, dans le code de la partie précédente, l'implémentation manuelle de la rétropropagation par la fonctionnalité *autograd*. Ainsi :
* la méthode `def backward(self,dc_dS, S, X2, X1, X0)` de la classe `class MLP` doit être supprimée, et l'appel à cette méthode remplacé par l'appel à la méthode `.backward()` de l'autograd comme vu précédemment (`S.backward(dc_dS)`),
* il faut activer le calcul des gradients des paramètres `W1`, `b1`, etc. (méthode `.requires_grad_()`), supprimer les variables gradients `dc_dW1`, `dc_db1`, etc.  
* il faut modifier `GradientDescentWithMomentum` pour qu'il utilise les champs `.grad` des paramètres (car les variables gradients `dc_dW1`, `dc_db1`, etc. n'existent plus), et mettre au début de la méthode `def step(self):` `with t.no_grad():` pour que les opérations qui suivent ne soient pas mises dans le graphe de calcul.


Le code obtenu devrait fonctionner exactement comme dans la partie précédente.  
Si vous êtes bloqués, vous pouvez vous référez à cette correction : [main_MLP_two_layers_pytorch_with_autograd.py](https://gbourmaud.github.io/files/intro_deep_learning/TP/TP_MLP/misc/main_MLP_two_layers_pytorch_with_autograd.py).

## IV) Utilisation du paquet `torch.nn`

En plus de la fonctionnalité autograd, la bibliothèque PyTorch contient de nombreuses implémentations de fonctions paramétriques qui permettent de construire une architecture beaucoup plus rapidement que ce que nous avons fait jusqu'à présent. Ces fonctions se trouvent dans le paquet `torch.nn`.

### Exemple de la transformation affine générale ("Fully Connected")

In [ ]:
import torch.nn as nn
import torch

linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

x = torch.randn(10, 3)
pred = linear(x)

On remarque que cette fonctionnalité "cache" beaucoup de détails d'implémentation. Par exemple concernant la fonction `nn.linear`, ses paramètres sont définis implicitement ainsi que la méthode d'initialisation de leurs valeurs.

En modifiant la classe `class MLP` (de l'implémentation utilisant l'autograd) en faisant usage du paquet `torch.nn` on obtient l'implémentation suivante.

In [ ]:
class MLP(nn.Module):
    def __init__(self, H):
        super(MLP, self).__init__()
        
        self.C = 3
        self.D = 2
        self.H = H
        
        self.fc1 = nn.Linear(self.D, self.H) 
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(self.H, self.C)  
        
        #init parameters
        with t.no_grad():
            self.fc1.weight.uniform_(-math.sqrt(6./self.D), math.sqrt(6./self.D))
            self.fc1.bias.uniform_(-1./math.sqrt(self.D), 1./math.sqrt(self.D))
            self.fc3.weight.uniform_(-math.sqrt(6./self.H),math.sqrt(6./self.H))
            self.fc3.bias.uniform_(-1./math.sqrt(self.H),1./math.sqrt(self.H))
        
    def forward(self,X):
    
        X1 = self.fc1(X) #NxH
        X2 = self.relu(X1) #NxH
        S = self.fc3(X2) #NxC
    
        return X,X1,X2,S

Parmi les fonctions disponibles, `torch.nn` contient également les fonctions de coûts les plus communément utilisées. Ainsi la fonction `multinoulliCrossEntropyLoss` peut être remplacée par son équivalent PyTorch `nn.CrossEntropyLoss`.

Le code obtenu devrait désormais être grandement simplifié par rapport au code initial utilisant Numpy, mais devrait fonctionner exactement comme auparavant.  
Si vous êtes bloqués, vous pouvez vous référez à cette correction : [main_MLP_two_layers_pytorch_with_nn.py](https://gbourmaud.github.io/files/intro_deep_learning/TP/TP_MLP/misc/main_MLP_two_layers_pytorch_with_nn.py).

## V) Utilisation du paquet `torch.optim`

Plusieurs algorithmes d'optimisation sont également disponibles dans le paquet `torch.optim`. 

Lire la page de la documentation concernant ce paquet https://pytorch.org/docs/stable/optim.html?highlight=torch%20optim#module-torch.optim.

Utiliser l'algorithme `torch.optim.SGD` pour simplifier le code précédent.

Si vous êtes bloqués, vous pouvez vous référez à cette correction : [main_MLP_two_layers_pytorch_with_optim.py](https://gbourmaud.github.io/files/intro_deep_learning/TP/TP_MLP/misc/main_MLP_two_layers_pytorch_with_optim.py).  
  
Observer comme le code est beaucoup plus court par rapport au début du TP, mais un certain nombre de choses sont désormais cachées. Désormais vous savez ce qui se cache dedans !